In [116]:
import numpy as np
from numpy import random as rnd
from matplotlib import pyplot as plt
import datetime,os

import torch
import pandas as pd

from fuzzywuzzy import fuzz
import re
from itertools import product

import ipywidgets as widgets 
from IPython.display import HTML

datapath = '..\\..\\..\\..\\TF_data\\ADMIN\\v2\\04_DATA\\'

In [117]:
# Load Account tracker data
df = pd.read_excel(datapath+'ATR.xlsx',index_col='Date',parse_dates=['Date'])

@widgets.interact(lookback_months=widgets.IntSlider(min=1,max=df.shape[0],step=1,value=12,description='Months back '))
def plt_acc_tracker(lookback_months):
    
    fig = plt.figure(figsize=[17,7])

    t_df = df[df.index.isin(df.index.tolist()[-lookback_months:])]
    x = t_df.index

    t0 = t_df['Min Month'].values
    t1 = t_df['Max Month'].values

    plt.xticks(rotation=75)

    plt.fill_between(x,t0,t1,alpha=0.2)
    plt.plot(t_df['Account Total'])

    plt.grid()
    plt.xlabel('Time')
    plt.ylabel('Account Total [Euros]')
    plt.title('Account Tracker')

    plt.show()
    return

interactive(children=(IntSlider(value=12, description='Months back ', max=15, min=1), Output()), _dom_classes=…

In [118]:
# Load running monthly aggregates
m0 = pd.read_excel(datapath+'RunningMonthlyExp.xlsx',sheet_name='p0',index_col=0)
m1 = pd.read_excel(datapath+'RunningMonthlyExp.xlsx',sheet_name='p1',index_col=0)
m2 = pd.read_excel(datapath+'RunningMonthlyExp.xlsx',sheet_name='p2',index_col=0)
m3 = pd.read_excel(datapath+'RunningMonthlyExp.xlsx',sheet_name='p3',index_col=0)

@widgets.interact(selected=widgets.SelectMultiple(options=['Curr.month', 'Curr.month-1', 'Curr.month-2','Curr.month-3'],value=['Curr.month', 'Curr.month-1', 'Curr.month-2','Curr.month-3'],description='Show Months '))
def plot_prev_monthly_aggregates(selected):
    
    fig = plt.figure(figsize=(17,7))

    if 'Curr.month' in selected: plt.plot(m0[:pd.to_datetime(datetime.datetime.today()).day].cumsum().values,'b')
    if 'Curr.month-1' in selected: plt.plot(m1.cumsum().values,'g--')
    if 'Curr.month-2' in selected: plt.plot(m2.cumsum().values,'r--')
    if 'Curr.month-3' in selected: plt.plot(m3.cumsum().values,'y--')

    plt.xlabel('Days in Month')
    plt.ylabel('Cumulative Spending [Euros]')
    plt.title('Expenses: Comparison {} to previous Months'.format(pd.to_datetime(datetime.datetime.today()).month_name()))

    plt.legend(selected)
    plt.grid()

    plt.show()
    return

interactive(children=(SelectMultiple(description='Show Months ', index=(0, 1, 2, 3), options=('Curr.month', 'C…

In [119]:
# Load monthly aggregates
mf = pd.read_excel(datapath+'MonthlyExpRevAggregate.xlsx',index_col=0)

@widgets.interact(lookback_months=widgets.IntSlider(min=1,max=mf.shape[0],step=1,value=6,description='Months back '))
def plot_exp_rev(lookback_months):
    
    # Plot bar chart
    ax = mf[mf.index.isin(mf.index.tolist()[-lookback_months:])].plot(kind='bar',rot=75,figsize=(17,7),grid=True,color=['r','g'])
    
    ax.set_title('Expenses and Revenue')
    ax.set_xlabel('Date')
    ax.set_ylabel('Euros')
    
    plt.show()
    return

interactive(children=(IntSlider(value=6, description='Months back ', max=16, min=1), Output()), _dom_classes=(…

In [120]:
# Load single transactions file
st = pd.read_excel(datapath+'STF.xlsx')

@widgets.interact(lookback_months=widgets.IntSlider(min=1,max=24,step=1,value=12,description='Months back '))
def plot_posts(lookback_months):

    # Set start and end date
    end_date = pd.to_datetime(datetime.datetime.today().strftime('%Y-%m-%d'))
    start_date = end_date - pd.DateOffset(months=lookback_months)

    # Aggregate target data
    exp_stf = st[(st['Value_transaction']<=0)&(st['Date_ordered']>=start_date)&(st['Date_ordered']<end_date)][['ExpenseGroup','Value_transaction']].groupby(['ExpenseGroup']).sum()
    exp_stf['Value_transaction'] = exp_stf['Value_transaction'].abs()

    rev_stf = st[(st['Value_transaction']>0)&(st['Date_ordered']>=start_date)&(st['Date_ordered']<end_date)][['ExpenseGroup','Value_transaction']].groupby(['ExpenseGroup']).sum()
    rev_stf['Value_transaction'] = rev_stf['Value_transaction'].abs()

    # Make plot
    fig, ax = plt.subplots(1,2,figsize=[17,5])

    rev_stf.sort_values(by=['Value_transaction'],ascending=False).plot(kind='bar',ax=ax[0],legend=False,color='g',logy=True)
    exp_stf.sort_values(by=['Value_transaction'],ascending=False).plot(kind='bar',ax=ax[1],legend=False,color='r',logy=True)

    ax[0].grid()
    ax[0].set_xlabel('')
    ax[0].set_ylabel('Euros (log)')
    for tick in ax[0].get_xticklabels(): tick.set_rotation(-45)
    ax[0].set_title('Revenue')
    #for p in ax[0].patches: ax[0].annotate(str(int(p.get_height()))+'EUR', (p.get_x(), p.get_height()*1.2))

    ax[1].grid()
    ax[1].set_xlabel('')
    ax[1].set_ylabel('Euros (log)')
    for tick in ax[1].get_xticklabels(): tick.set_rotation(-45)
    ax[1].set_title('Expenses')
    #for p in ax[1].patches: ax[1].annotate(str(int(p.get_height()))+'EUR', (p.get_x(), p.get_height()*1.2))

    plt.show()
    return

interactive(children=(IntSlider(value=12, description='Months back ', max=24, min=1), Output()), _dom_classes=…

In [121]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')